# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.


### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

****

In [2]:
import pandas as pd
import numpy as np
from scipy import stats
from matplotlib import pyplot as plt

In [3]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [4]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [5]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>Since we are dealing with categorical data and want to know whether the category is independent of the result (the call back), a Chi-squared test of independence is appropriate to use. The central limit theorem does apply as the sample sizes are greater than 30. For this situation, the null hypothesis is that the call back rate is the same regardless of assumed race, while the alternative hypothesis is that the assumed race has some sort of influence on call backs.</p>
</div>

In [6]:
w = data[data.race=='w']
b = data[data.race=='b']

In [7]:
# number of callbacks for white-sounding names
sum(data[data.race=='w'].call)

235.0

# Frequentist Testing

In [8]:
# Your solution to Q3 here

app_tab = pd.crosstab(data.race, data.call, margins = True)

app_tab.columns = ["0","1","row_totals"]

app_tab.index = ["b","w","col_totals"]

observed = app_tab.iloc[0:2,0:2]   # Get table without totals for later use
app_tab


,0,1,row_totals
b,2278,157,2435
w,2200,235,2435
col_totals,4478,392,4870


In [9]:
expected =  np.outer(app_tab["row_totals"][0:2],
                     app_tab.loc["col_totals"][0:2]) / 4870

expected = pd.DataFrame(expected)

expected.columns = ["0","1"]
expected.index = ["b","w"]

expected

,0,1
b,2239.0,196.0
w,2239.0,196.0


In [10]:
chi_squared_stat = (((observed-expected)**2)/expected).sum().sum()

print(chi_squared_stat)

16.87905041427022


In [11]:
crit = stats.chi2.ppf(q = 0.95, # Find the critical value for 95% confidence*
                      df = 8)   # *

print("Critical value")
print(crit)

p_value = 1 - stats.chi2.cdf(x=chi_squared_stat,  # Find the p-value
                             df=1)
print("P value")
print(p_value)

Critical value
15.50731305586545
P value
3.983886837588546e-05


In [12]:
stats.chi2_contingency(observed= observed)

(16.44902858418937, 4.997578389963255e-05, 1, array([[2239.,  196.],
        [2239.,  196.]]))

# Bootstrapping

In [13]:
# Construct arrays of data: callbacks, white-sounding names, black-sounding names
cb = np.sum(data.call)
datalen = len(data)
p = cb/datalen

wcb = np.sum(w.call)
wlen = len(w)
wp = wcb/wlen

bcb = np.sum(b.call)
blen = len(b)
bp = bcb/blen


all_cb = np.array([True] * int(cb) + [False] * int(datalen-cb))

size = 10000

bsreps_diff = np.empty(size)

#Generate replicates
for i in range(size):
    w_bs_replicates = np.sum(np.random.choice(all_cb, size=wlen))
    b_bs_replicates = np.sum(np.random.choice(all_cb, size=blen))
    
    bsreps_diff[i] = (w_bs_replicates - b_bs_replicates)/blen

#Get p-value
bs_pval = np.sum(bsreps_diff >= (wp-bp)) / len(bsreps_diff)

#Get confidence interval
bs_confint = np.percentile(bsreps_diff, [2.5, 97.5])
bs_avg_diff = np.mean(bsreps_diff)

print('obs diff: {}\n'.format(wp-bp))
print('p-value: {}\n95% conf. int.: {}'.format(bs_pval, bs_confint))

obs diff: 0.032032854209445585

p-value: 0.0
95% conf. int.: [-0.01478439  0.01519507]


<div class="span5 alert alert-success">
<p> Given that p-value is less than 0.05, the assumed race from an applicant's name does affect whether they are called for an interview. While the assumed race is shown to be a statistically significant factor, there are still other variables that weren't considered in the scope of this analysis. This specific analysis is looking at whether the assumed racial identity from a name affects callback success but it not indicative of callback success in general. Generalizing the rate of callback would require checking for statistical significance of other variables such as years of job experience, education, etc.     </p>
</div>